In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import cv2
import os

flipped_path = r'E:\cyshield\heatmap classification\training\right'
not_flipped_path = r'E:\cyshield\heatmap classification\training\wrong'

validation_right_path = r'E:\cyshield\heatmap classification\validation\right'
validation_wrong_path = r'E:\cyshield\heatmap classification\validation\wrong'
def load_and_preprocess_data(image_path, label):
    images = []
    labels = []

    for filename in os.listdir(image_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(image_path, filename))
            img = cv2.resize(img, (100, 100))
            images.append(img)
            labels.append(label)

    return images, labels

flipped_images, flipped_labels = load_and_preprocess_data(flipped_path, 'flipped')
not_flipped_images, not_flipped_labels = load_and_preprocess_data(not_flipped_path, 'not_flipped')

all_images = flipped_images + not_flipped_images
all_labels = flipped_labels + not_flipped_labels

label_encoder = LabelEncoder()
all_labels_encoded = label_encoder.fit_transform(all_labels)

all_images, all_labels_encoded = shuffle(all_images, all_labels_encoded, random_state=42)

val_right_images, val_right_labels = load_and_preprocess_data(validation_right_path, 'flipped')

val_wrong_images, val_wrong_labels = load_and_preprocess_data(validation_wrong_path, 'not_flipped')

val_images = val_right_images + val_wrong_images
val_labels = val_right_labels + val_wrong_labels

val_labels_encoded = label_encoder.transform(val_labels)

all_images = np.array(all_images)/255.0
val_images = np.array(val_images)/ 255.0

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(all_images, all_labels_encoded, epochs=10, validation_data=(val_images, val_labels_encoded))
val_loss, val_acc = model.evaluate(val_images, val_labels_encoded)
print(val_acc)
model.save('flipped_detection_model.h5')
